In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

import torch
import torch.optim as optim

from scipy.spatial import ConvexHull
from matplotlib.patches import Polygon
import sys,os
sys.path.append(os.path.realpath('./src/'))
import pandas as pd
from materialEncoder import MaterialEncoder
import time
import itertools
%matplotlib qt

In [2]:
def to_np(x):
  return x.detach().cpu().numpy()

def to_torch(x):
  return torch.tensor(x).float()

In [3]:
def preprocessData():
  df = pd.read_excel('./data/solidworksMaterialDatabase.xlsx')# #  #subsetSolidworksDatabase
  dataIdentifier = {'name': df[df.columns[0]], 'className':df[df.columns[1]], 'classID':df[df.columns[2]]} # name of the material and type
  trainInfo = np.log10(df[df.columns[3:]].to_numpy())
  dataScaleMax = torch.tensor(np.max(trainInfo, axis = 0))
  dataScaleMin = torch.tensor(np.min(trainInfo, axis = 0))
  normalizedData = (torch.tensor(trainInfo) - dataScaleMin)/(dataScaleMax - dataScaleMin)
  # trainingData = torch.tensor(normalizedData).float()
  trainingData = normalizedData.clone().detach().float().requires_grad_(True)

  dataInfo = {'youngsModulus':{'idx':0,'scaleMin':dataScaleMin[0], 'scaleMax':dataScaleMax[0]},\
              'costPerKg':{'idx':1,'scaleMin':dataScaleMin[1], 'scaleMax':dataScaleMax[1]},\
              'massDensity':{'idx':2,'scaleMin':dataScaleMin[2], 'scaleMax':dataScaleMax[2]},\
              'yieldStrength':{'idx':3,'scaleMin':dataScaleMin[3], 'scaleMax':dataScaleMax[3]}}
  return trainingData, dataInfo, dataIdentifier, trainInfo
trainingData, dataInfo, dataIdentifier, trainInfo = preprocessData()
numMaterialsInTrainingData, numFeatures = trainingData.shape

In [4]:
latentDim, hiddenDim = 2, 250
numEpochs = 50000
klFactor = 5e-5
learningRate = 2e-3
savedNet = './data/vaeNet.nt'
vaeSettings = {'encoder':{'inputDim':numFeatures, 'hiddenDim':hiddenDim,\
                                          'latentDim':latentDim},\
               'decoder':{'latentDim':latentDim, 'hiddenDim':hiddenDim,\
                                          'outputDim':numFeatures}}

materialEncoder = MaterialEncoder(trainingData, dataInfo, dataIdentifier, vaeSettings)

# start = time.perf_counter()
convgHistory = materialEncoder.trainAutoencoder(numEpochs, klFactor, savedNet, learningRate)
# print('training time : {:.2F} '.format(time.perf_counter() - start))


Iter 0 reconLoss 3.04E+01 klLoss 1.40E-02 loss 3.04E+01
Iter 500 reconLoss 4.10E-01 klLoss 7.92E-02 loss 4.89E-01
Iter 1000 reconLoss 1.93E-01 klLoss 7.19E-02 loss 2.65E-01
Iter 1500 reconLoss 1.49E-01 klLoss 6.65E-02 loss 2.16E-01
Iter 2000 reconLoss 1.17E-01 klLoss 6.37E-02 loss 1.81E-01
Iter 2500 reconLoss 9.70E-02 klLoss 6.18E-02 loss 1.59E-01
Iter 3000 reconLoss 8.06E-02 klLoss 6.08E-02 loss 1.41E-01
Iter 3500 reconLoss 6.84E-02 klLoss 5.96E-02 loss 1.28E-01
Iter 4000 reconLoss 6.23E-02 klLoss 5.85E-02 loss 1.21E-01
Iter 4500 reconLoss 5.98E-02 klLoss 5.70E-02 loss 1.17E-01
Iter 5000 reconLoss 5.44E-02 klLoss 5.64E-02 loss 1.11E-01
Iter 5500 reconLoss 5.29E-02 klLoss 5.58E-02 loss 1.09E-01
Iter 6000 reconLoss 4.76E-02 klLoss 5.50E-02 loss 1.03E-01
Iter 6500 reconLoss 4.79E-02 klLoss 5.42E-02 loss 1.02E-01
Iter 7000 reconLoss 4.47E-02 klLoss 5.37E-02 loss 9.84E-02
Iter 7500 reconLoss 4.86E-02 klLoss 5.34E-02 loss 1.02E-01
Iter 8000 reconLoss 4.09E-02 klLoss 5.26E-02 loss 9.35E-02
I

In [5]:
# print(materialEncoder.vaeNet.encoder.z)

In [6]:
# materialEncoder.loadAutoencoderFromFile('./results/vaeTrained.pkl')

In [62]:
from examples import getExample
from truss3DFE import Truss3DFE

exampleName, nodeXY, connectivity, bc = getExample(19)
bc['forces'] = {'nodes':np.array([8]), 'fx':1.E3*torch.tensor([0.]), 'fy':1E2*torch.tensor([200.]), 'fz':1E0*torch.tensor([0.])}
truss = Truss3DFE(nodeXY, connectivity, bc)


A = 2e1*torch.ones((connectivity.shape[0]))
u, dispX, dispY, dispZ, nodalDeformation, internalForce = truss.solveFE(E = torch.tensor([2e11]).float(), A =A)
truss.plot(f'J {truss.computeCompliance(u):.2E} , volume {truss.getVolume(A):.2E}', plotDeformed = True)

In [46]:
# lossMethod = {'type':'penalty', 'alpha0':0.05, 'delAlpha':0.15}
lossMethod = {'type':'logBarrier', 't0':3, 'mu':1.001}
optParams = {'minEpochs':150, 'maxEpochs':1000, 'lossMethod':lossMethod,'lr':0.001,'gradclip':{'isOn':False, 'thresh':2e-1}}

In [63]:
plt.close('all')

import torch
import torch.optim as optim
import numpy as np
import itertools
import matplotlib.pyplot as plt
from networks import TopologyNetwork, MaterialNetwork
from utilFuncs import to_torch
import torch.nn as nn
lkyReLU = nn.LeakyReLU(0.00)

class Truss3DOptimization:
  def __init__(self, truss, materialEncoder):
    self.truss = truss
    self.materialEncoder = materialEncoder
    self.verbose = True
  #--------------------------#
  def computeMetrics(self, area, properties):
    metrics = {}
    u, _, _, _, _, internalForce = self.truss.solveFE(properties['youngsModulus'], area)
    metrics['compliance'] = self.truss.computeCompliance(u)
    metrics['volume'] = self.truss.getVolume(area)
    metrics['mass'] = properties['massDensity']*metrics['volume']
    metrics['cost'] = metrics['mass']*properties['costPerKg']
    return metrics, u, internalForce
  #--------------------------#
  def computeConstraints(self, constraints, metrics, area, internalForce, properties):
    #~~~~~~~~~~~#
    def pnormMax(x, p = 6.):
      return torch.pow(torch.sum(x**p), 1./p)
    #~~~~~~~~~~~#
    if(constraints['massCons']['isOn']):
      constraints['massCons']['value'] = (metrics['mass']/constraints['massCons']['maxMass']) - 1.
    #~~~~~~~~~~~#
    if(constraints['costCons']['isOn']):
      constraints['costCons']['value'] = (metrics['cost']/constraints['costCons']['maxCost']) - 1.
    #~~~~~~~~~~~#
    if(constraints['volumeCons']['isOn']):
      constraints['volumeCons']['value'] = \
        (metrics['volumeCons']/constraints['volumeCons']['desiredVolume']) - 1.
    #~~~~~~~~~~~#
    if(constraints['tensionCons']['isOn']):
      tensileForce = 0.001+torch.relu(internalForce)
      tensileStress = tensileForce/area
      maxTensileStressExperienced = pnormMax(tensileStress)*constraints['tensionCons']['FOS']
      constraints['tensionCons']['value'] = (maxTensileStressExperienced/properties['yieldStrength']) - 1.
    #~~~~~~~~~~~#
    if(constraints['compressionCons']['isOn']):
      compressiveForce = 0.001+torch.relu(-internalForce)
      k = 4. # used in geom factor... see buckling theory
      geomFactor = k*(self.truss.barLength/(np.pi*area))**2
      compressiveStress = compressiveForce*geomFactor
      maxCompressiveStressExperienced = pnormMax(compressiveStress)*constraints['compressionCons']['FOS']
      constraints['compressionCons']['value'] = \
        (maxCompressiveStressExperienced/properties['youngsModulus']) - 1.
    return constraints     
  #--------------------------#
  def optimizeDesign(self, optParams, areaOptimization, materialOptimization, constraints):
    #~~~~~~~~~~~#
    def computeConstraintTerm(consVal):
      if(optParams['lossMethod']['type'] == 'penalty'):
        alpha = min(100.,optParams['lossMethod']['alpha0'] + \
                epoch*optParams['lossMethod']['delAlpha']) # penalty method
        consTerm = alpha*lkyReLU(consVal)#**2
      if(optParams['lossMethod']['type'] == 'logBarrier'):
        t = optParams['lossMethod']['t0']* \
                          optParams['lossMethod']['mu']**epoch;
        if(consVal < (-1/t**2)):
          consTerm = -torch.log(-consVal)/t
        else:
          consTerm = t*consVal - np.log(1/t**2)/t + 1./t
      return consTerm
    #~~~~~~~~~~~#
    def rescale(val, maxVal, minVal):
      return 10.**(minVal + val*(maxVal - minVal))
    #~~~~~~~~~~~#
    designVariables = []
    if(areaOptimization['isOn']):
      topNet = TopologyNetwork(areaOptimization['netSettings'])
      xyBarCenter = to_torch(self.truss.barCenter)
      designVariables = itertools.chain(designVariables,topNet.parameters())
    if(materialOptimization['isOn']):
      matNet = MaterialNetwork(materialOptimization['netSettings'])
      matInput = torch.tensor([1.], requires_grad = True).float().view(-1,1)
      designVariables = itertools.chain(designVariables,matNet.parameters())
    #~~~~~~~~~~~#
    metrics = {'compliance':0., 'volume':0., 'mass':0., 'cost':0.}
    convergenceHistory = {} # keep track of metrics and constraints
    for k in metrics:
      convergenceHistory[k] = []
    for c in constraints:
      convergenceHistory[c] = []
    obj0 = 1.

    # optimizer = optim.AdamW(designVariables, amsgrad=True, lr=optParams['lr'], eps = 1e-3)
    optimizer = optim.Adagrad(designVariables, lr=0.01)
    #~~~~~~~~~~~#
    print("iter \t J \t\t vol \t\t mass \t\t cost")
    for epoch in range(optParams['maxEpochs']):
      #~~~~~~~~~~~#
      def getMaterialProperties(decoded):
        properties = {}
        keys = ['youngsModulus', 'costPerKg', 'massDensity', 'yieldStrength']
        dataInfo = self.materialEncoder.dataInfo # too lazy to write :D
        for k in keys:
          properties[k] = rescale(decoded[:,dataInfo[k]['idx']],\
                          dataInfo[k]['scaleMax'],\
                          dataInfo[k]['scaleMin'])
        return properties
      #~~~~~~~~~~~#
      if(areaOptimization['isOn']):
        
        nnOut = topNet(xyBarCenter)
        self.area = areaOptimization['bounds']['min'] + \
          (areaOptimization['bounds']['max'] - \
           areaOptimization['bounds']['min'])*nnOut
      else:
        self.area = areaOptimization['area']
      #~~~~~~~~~~~#
      if(materialOptimization['isOn']):
        self.optimalZ = matNet(matInput)
        decoded = self.materialEncoder.vaeNet.decoder(self.optimalZ) 
        properties = getMaterialProperties(decoded)
      else:
        properties = materialOptimization['properties']
      #~~~~~~~~~~~#
      metrics, u, internalForce = self.computeMetrics(self.area, properties)
      #~~~~~~~~~~~#
      constraints = self.computeConstraints(constraints, metrics, self.area, internalForce, properties)
      #~~~~~~~~~~~#
      loss = (metrics['compliance']/obj0)
      for c in constraints:
        if(constraints[c]['isOn']):
          loss = loss + computeConstraintTerm(constraints[c]['value'])
          convergenceHistory[c].append(constraints[c]['value'].item())
      #~~~~~~~~~~~#
      if(epoch == 0):
        obj0 = metrics['compliance'].item()
      #~~~~~~~~~~~#
      loss.backward(retain_graph=True)
#       if(areaOptimization['isOn'] and optParams['gradclip']['isOn']):
#         torch.nn.utils.clip_grad_norm_(topNet.parameters(),optParams['gradclip']['thresh'])
      if(materialOptimization['isOn'] and optParams['gradclip']['isOn']):
        torch.nn.utils.clip_grad_norm_(matNet.parameters(),optParams['gradclip']['thresh'])
      optimizer.step()
      #~~~~~~~~~~~#
      titleStr = f" {epoch} \t {metrics['compliance'].item():.2E} \t \
         {metrics['volume'].item():.2E} \t {metrics['mass'].item():.2E} \t \
         {metrics['cost'].item():.2E}"
      if(self.verbose and epoch%10 == 0):
        print(titleStr)
      #~~~~~~~~~~~#
      for c in constraints:
        if(constraints[c]['isOn']):
          convergenceHistory[c].append(constraints[c]['value'].item())
      for k in metrics:
        convergenceHistory[k].append(metrics[k].item())
      #~~~~~~~~~~~#
      if(epoch%50 ==0 and self.verbose):
        self.truss.plot(f"iter {epoch}, J {metrics['compliance'].item():.2E}", False)
        plt.pause(0.001)
    print(titleStr)
    print('-------\n--constraints------\n')
    for c in constraints:
      if(constraints[c]['isOn']):
        print(f" {c} : {constraints[c]['value'].item():.2E}")
    print('---------------\n')
    return convergenceHistory

In [105]:
topNetSettings = {'inputDim':3, 'numLayers':3, 'numNeuronsPerLyr':20, 'outputDim':1}
matNetSettings = {'inputDim':1, 'numLayers':2,\
                  'numNeuronsPerLyr':20, 'outputDim':latentDim, 'zMin':-2., 'zRange':4.}

constraints = {'volumeCons': {'isOn':False, 'desiredVolume':2e-6},\
               'massCons': {'isOn':True, 'maxMass':4E6}, \
               'costCons': {'isOn':False, 'maxCost':6e1},\
               'tensionCons': {'isOn':True, 'FOS':4.},\
               'compressionCons': {'isOn':True, 'FOS':4.}}
#for cost 900 iters

areaOptimization = {'isOn':True, 'bounds':{'min':2e1, 'max':1e2}, 'netSettings':topNetSettings}
# areaOptimization = {'isOn':False, 'area':2*torch.ones((connectivity.shape[0]), requires_grad = True)}

materialOptimization = {'isOn':True, 'netSettings':matNetSettings}
# materialOptimization = {'isOn':False, 'properties':{'youngsModulus':torch.tensor([1.e9]),\
#                                                     'massDensity':torch.tensor([1.e3]),\
#                                                     'yieldStrength':torch.tensor([1.e7]), \
#                                                     'costPerKg':torch.tensor([4.])}}

trussopt = Truss3DOptimization(truss, materialEncoder)

plt.close('all')
convergenceHistory = trussopt.optimizeDesign(optParams, areaOptimization, materialOptimization, constraints)

iter 	 J 		 vol 		 mass 		 cost
 0 	 9.66E-04 	          1.32E+03 	 3.83E+06 	          8.58E+06
 10 	 1.09E-03 	          1.09E+03 	 3.03E+06 	          7.35E+06
 20 	 7.15E-04 	          9.87E+02 	 3.38E+06 	          1.62E+07
 30 	 8.43E-04 	          8.54E+02 	 2.87E+06 	          1.23E+07
 40 	 7.78E-04 	          8.19E+02 	 2.94E+06 	          1.57E+07
 50 	 7.64E-04 	          7.81E+02 	 2.96E+06 	          1.81E+07
 60 	 7.95E-04 	          7.62E+02 	 2.81E+06 	          1.62E+07
 70 	 7.80E-04 	          7.68E+02 	 2.85E+06 	          1.64E+07
 80 	 7.58E-04 	          7.70E+02 	 2.93E+06 	          1.79E+07
 90 	 7.75E-04 	          7.79E+02 	 2.84E+06 	          1.57E+07
 100 	 7.78E-04 	          7.81E+02 	 2.82E+06 	          1.53E+07
 110 	 7.54E-04 	          7.94E+02 	 2.91E+06 	          1.61E+07
 120 	 7.63E-04 	          7.94E+02 	 2.87E+06 	          1.54E+07
 130 	 7.66E-04 	          8.06E+02 	 2.85E+06 	          1.46E+07
 140 	 7.52E-04 	          8.28E+02 	 2.9

In [102]:
print(trussopt.area)

tensor([20.0001, 20.0000, 20.0001, 20.0000, 20.0001, 20.0000, 20.0001, 20.0000,
        20.0001, 20.0000, 20.0001, 20.0001, 20.0002, 20.0006, 20.0002, 20.0001,
        20.0001, 20.0002, 20.0002, 20.0002], grad_fn=<AddBackward0>)


In [97]:
from utilFuncs import plotConvergence
plotConvergence(convergenceHistory)

In [106]:
matToUse = trussopt.materialEncoder.getClosestMaterialFromZ(trussopt.optimalZ, numClosest =3)
print(matToUse['material'][0])


ltnt1, ltnt2 = 0, 1
fig, ax = materialEncoder.plotLatent(ltnt1 = ltnt1, ltnt2 = ltnt2, plotHull = False, annotateHead = False,\
                                    saveFileName = './figures/optimalLatent.pdf')
zOpt_np = to_np(trussopt.optimalZ)
for i in range(zOpt_np.shape[0]):
  ax.annotate('OPTIMAL MATERIAL {:d}'.format(i+1), (zOpt_np[i,ltnt1], zOpt_np[i,ltnt2]))
  ax.scatter(zOpt_np[i,ltnt1], zOpt_np[i,ltnt2], marker='*',s=400)

closest material 0 : 2018 , confidence 86.02
closest material 1 : 2024-T361 , confidence 84.59
closest material 2 : 2014-T6 , confidence 83.71
2018


### Re-optimize with nearest material

In [107]:
from utilFuncs import to_torch
ind = np.where(dataIdentifier['name'] == matToUse['material'][0])
matData = 10.**trainInfo[ind,:].reshape(-1)
properties = {'name': matToUse['material'][0], \
               'youngsModulus':to_torch(matData[0]), \
               'costPerKg':to_torch(matData[1]), \
               'massDensity':to_torch(matData[2]), \
               'yieldStrength': to_torch(matData[3])}

print(properties)
areaOptimization = {'isOn':True, 'bounds':{'min':2e1, 'max':1e2}, 'netSettings':topNetSettings}
materialOptimization = {'isOn':False, 'properties':properties}
convergenceHistory = trussopt.optimizeDesign(optParams, areaOptimization, materialOptimization, constraints)

{'name': 2018, 'youngsModulus': tensor(7.4000e+10), 'costPerKg': tensor(2.7140), 'massDensity': tensor(2800.), 'yieldStrength': tensor(4.2051e+08)}
iter 	 J 		 vol 		 mass 		 cost
 0 	 9.09E-04 	          1.32E+03 	 3.70E+06 	          1.00E+07
 10 	 1.09E-03 	          1.07E+03 	 3.00E+06 	          8.14E+06
 20 	 1.11E-03 	          1.03E+03 	 2.87E+06 	          7.80E+06
 30 	 1.04E-03 	          1.07E+03 	 3.00E+06 	          8.14E+06


/filespace/s/ssridhara/Research/MaTruss_private/src/utilFuncs.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x).float()


 40 	 1.00E-03 	          1.09E+03 	 3.06E+06 	          8.29E+06
 50 	 9.86E-04 	          1.09E+03 	 3.05E+06 	          8.28E+06
 60 	 9.83E-04 	          1.08E+03 	 3.02E+06 	          8.19E+06
 70 	 9.70E-04 	          1.08E+03 	 3.03E+06 	          8.22E+06
 80 	 9.62E-04 	          1.08E+03 	 3.02E+06 	          8.20E+06
 90 	 9.47E-04 	          1.09E+03 	 3.04E+06 	          8.26E+06
 100 	 9.38E-04 	          1.09E+03 	 3.06E+06 	          8.30E+06
 110 	 9.33E-04 	          1.09E+03 	 3.06E+06 	          8.31E+06
 120 	 9.28E-04 	          1.10E+03 	 3.07E+06 	          8.33E+06
 130 	 9.22E-04 	          1.10E+03 	 3.09E+06 	          8.37E+06
 140 	 9.19E-04 	          1.10E+03 	 3.09E+06 	          8.39E+06
 150 	 9.19E-04 	          1.10E+03 	 3.09E+06 	          8.38E+06
 160 	 9.15E-04 	          1.11E+03 	 3.10E+06 	          8.42E+06
 170 	 9.14E-04 	          1.11E+03 	 3.10E+06 	          8.42E+06
 180 	 9.13E-04 	          1.11E+03 	 3.11E+06 	          8.43E+06
 

### Bruteforce

In [ ]:
trussopt.verbose = False
z = trussopt.materialEncoder.vaeNet.encoder.z.to('cpu').detach().numpy()
print('material \t z0 \t z1 \t J \t volume \t mass \t cost \t massCons \t tensionCons \t compressionCons ')
bruteProp = {'material':[], 'J':[], 'cost':[], 'mass':[], 'z0':[], 'z1':[], 'tensionCons':[], 'compressionCons':[]}
for ind in range(trainInfo.shape[0]):
  print(dataIdentifier['name'][ind], end = '\t')
  print(f"{z[ind,0]:.2E} \t {z[ind,1]:.2E}", end = '\t')
  matData = 10.**trainInfo[ind,:].reshape(-1)
  properties = {'name': matToUse['material'][0], \
                 'youngsModulus':to_torch(matData[0]), \
                 'costPerKg':to_torch(matData[1]), \
                 'massDensity':to_torch(matData[2]), \
                 'yieldStrength': to_torch(matData[3])}

  materialOptimization = {'isOn':False, 'properties':properties}
#   convergenceHistory = trussopt.optimizeDesign(optParams, areaOptimization, materialOptimization, constraints)
  metrics, u, internalForce = trussopt.computeMetrics(areaOptimization['area'],  properties)
  constraints = trussopt.computeConstraints(constraints, metrics, areaOptimization['area'], internalForce, properties)
  for k in metrics:
    print(f' {metrics[k].item():.2E}', end = '\t')
  for c in constraints:
    if(constraints[c]['isOn']):
      print(f"{constraints[c]['value'].item():.2E}", end = '\t')
  print('\n')
  bruteProp['material'].append(dataIdentifier['name'][ind])
  bruteProp['J'].append(metrics['compliance'].item())
  bruteProp['cost'].append(metrics['cost'].item())
  bruteProp['mass'].append(metrics['mass'].item())
  bruteProp['z0'].append(z[ind,0].item())
  bruteProp['z1'].append(z[ind,1].item())
  bruteProp['tensionCons'].append(constraints['tensionCons']['value'].item())
  bruteProp['compressionCons'].append(constraints['compressionCons']['value'].item())

In [ ]:
def plotLatent(prop, cutoff):
    clrs = ['purple', 'green', 'orange', 'pink', 'yellow', 'black', 'violet', 'cyan', 'red', 'blue']
    colorcol = dataIdentifier['classID']
    ptLabel = dataIdentifier['name']
    autoencoder = trussopt.materialEncoder.vaeNet
    z = autoencoder.encoder.z.to('cpu').detach().numpy()
    fig, ax = plt.subplots()
    a = ax.scatter(z[:, 0], z[:, 1], marker='o', s = 16, c = prop, cmap = 'rainbow')
    plt.colorbar(a)
    for i in range(np.max(colorcol)+1): 
      zMat = np.vstack((z[colorcol == i,0], z[colorcol == i,1])).T #pts only in a certain class
      if(i == np.max(colorcol)): #removed for last class TEST
        break # END TEST
      hull = ConvexHull(zMat)
      cent = np.mean(zMat, 0)
      pts = []
      for pt in zMat[hull.simplices]:
          pts.append(pt[0].tolist())
          pts.append(pt[1].tolist())

      pts.sort(key=lambda p: np.arctan2(p[1] - cent[1],
                                      p[0] - cent[0]))
      pts = pts[0::2]  # Deleting duplicates
      pts.insert(len(pts), pts[0])
      poly = Polygon(1.1*(np.array(pts)- cent) + cent,
                     facecolor= 'none', alpha=1, edgecolor = 'black') #'black'
      poly.set_capstyle('round')
      plt.gca().add_patch(poly)
      ax.annotate(dataIdentifier['className'][i], (cent[0], cent[1]), size = 12)

    plt.xlabel('$z_0$', size = 18)
    plt.ylabel('$z_1$', size = 18)
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    return fig, ax
  
fig, ax = plotLatent(np.log10(bruteProp['J']), 0.)
ax.set_title('compliance')
plt.annotate('Opt. mat.', (zOpt_np[0,ltnt1], zOpt_np[0,ltnt2]))
plt.scatter(zOpt_np[0,0], zOpt_np[0,1], marker='*',s=400)

fig, ax = plotLatent(np.log10(bruteProp['mass']), np.log10(5e1) )
ax.set_title('mass')
plt.annotate('Opt. mat.', (zOpt_np[0,ltnt1], zOpt_np[0,ltnt2]))
plt.scatter(zOpt_np[0,0], zOpt_np[0,1], marker='*',s=400)

fig, ax = plotLatent(bruteProp['tensionCons'], 0.01)
ax.set_title('tension constraint')
plt.annotate('Opt. mat.', (zOpt_np[0,ltnt1], zOpt_np[0,ltnt2]))
plt.scatter(zOpt_np[0,0], zOpt_np[0,1], marker='*',s=400)

fig, ax = plotLatent(np.log10(bruteProp['cost']), 0.)
ax.set_title('cost')
plt.annotate('Opt. mat.', (zOpt_np[0,ltnt1], zOpt_np[0,ltnt2]))
plt.scatter(zOpt_np[0,0], zOpt_np[0,1], marker='*',s=400)

fig, ax = plotLatent(bruteProp['compressionCons'], 0.)
ax.set_title('compression constraint')
plt.annotate('Opt. mat.', (zOpt_np[0,ltnt1], zOpt_np[0,ltnt2]))
plt.scatter(zOpt_np[0,0], zOpt_np[0,1], marker='*',s=400)

In [ ]:
plt.close('all')